In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV , RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取酒類資料集
wine = datasets.load_wine()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)

# 建立模型
reg = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 8.91 的 MSE(基八大)

reg.fit(x_train , y_train)
y_pred = reg.predict(x_test)
print(metrics.mean_squared_error(y_test , y_pred))

0.14672894294872418


In [14]:
# 設定要訓練的超參數組合

n_estimators = [50, 100]
max_depth = [1, 3]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)

# grid_search = GridSearchCV(reg , param_grid , scoring = "neg_mean_squared_error" , n_jobs=-1, verbose=1)
grid_search1 = RandomizedSearchCV(reg , param_grid , scoring = "neg_mean_squared_error" , n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search1.fit(x_train , y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

C:\Users\lincecum\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\lincecum\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished
C:\Users\lincecum\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [15]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s " % (grid_result.best_score_ , grid_result.best_params_))

Best Accuracy: -0.081166 using {'n_estimators': 100, 'max_depth': 1} 


In [16]:
grid_result.best_params_

{'max_depth': 1, 'n_estimators': 100}

In [17]:
#　使用最佳參數重新建立模型

reg_bestparam = GradientBoostingRegressor(max_depth = grid_result.best_params_['max_depth'],
                                         n_estimators = grid_result.best_params_['n_estimators'])

#訓練模型
reg_bestparam.fit(x_train , y_train)

#預測測試值
y_pred = reg_bestparam.predict(x_test)

In [18]:
# 調整參數後降了0.2 屌
print(metrics.mean_squared_error(y_test , y_pred))

0.05221161654859812
